In [2]:
import torch
import torch.functional as F

import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import numpy as np

import matplotlib.pyplot as plt

device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class spikingNeuron(nn.Module):
    def __init__(self, in_features: int, out_features: int, device=None, dtype=None):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.storage = torch.zeros(self.in_features)
        self.weights = nn.Parameter(torch.empty(
            self.in_features, self.out_features, **factory_kwargs))
        self.bias = nn.Parameter(torch.empty(
            self.out_features, **factory_kwargs))

        nn.init.xavier_uniform_(self.weights)
        nn.init.normal_(self.bias, mean=0.0, std=1.0)

    def forward(self, x, t):

        # input x is binary vector
        x = x + self.storage
        self.storage = x
        x = torch.matmul(self.storage/t, self.weights) + self.bias
        x = F.relu(x)
        x = torch.clip(x, 0, 1)
        x = torch.bernoulli(x)

        return x